# Converting PyTorch to TensorFlow Lite for xCORE

## Using ONNX

ONNX is an open format built to represent machine learning models. We can convert from PyTorch to ONNX, then from ONNX to TensorFlow, then from TensorFlow to TensorFlow Lite, and finally, run it through xformer to optimise it for xCORE.

In [1]:
!pip install torch
!pip install tensorflow
!pip install -U onnx
!pip install -U nvidia-pyindex
!pip install -U onnx-graphsurgeon
!pip install -U polygraphy
!pip install -U onnxruntime
!pip install -U onnxsim
!pip install -U simple_onnx_processing_tools
!pip install -U onnx2tf
!pip install -U protobuf==3.20.3
!pip install -U h5py==3.7

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 41 kB 2.6 MB/s  eta 0:00:01


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 291 kB 3.9 MB/s eta 0:00:01     |████▌                           | 40 kB 1.8 MB/s eta 0:00:01
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 6.6 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 48.2 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 47.9 MB/s eta 0:00:01
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 3.2 MB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 239 kB 30.8 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 27.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 47.1 MB/s eta 0:00:01
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.8.1
    Uninstalling Pygments-2.8.1:
      Successfully uninstalled Pygments-2.8.1
ERROR: pi

### Import Model

For this example, we use mobilenet_v2.

In [2]:
import torch

model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

Using cache found in /Users/salmankhan/.cache/torch/hub/pytorch_vision_v0.10.0


### Convert to ONNX


In [3]:
batch_size = 8
channels = 3
height = 224
width = 224

sample_input = torch.rand((batch_size, channels, height, width))

onnx_model_path = "mobilenet_v2.onnx"

torch.onnx.export(
    model,
    sample_input,
    onnx_model_path,
    input_names=['input'],
    output_names=['output']
)

### Convert from ONNX to TensorFlow

Using unofficial package: https://github.com/PINTO0309/onnx2tf

In [4]:
import onnx2tf

tf_model_path = "mobilenet_v2.tf"

onnx2tf.convert(
    input_onnx_file_path=onnx_model_path,
    output_folder_path=tf_model_path,
    output_signaturedefs=True,
    non_verbose=True,
)

### Convert from TensorFlow to TFLite


In [6]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)

In [16]:
import numpy as np
def representative_dataset():
    batch_size = 8
    for _ in range(100):
      data = np.random.uniform(-0.1, 0.001, (batch_size, height, width, channels))
      yield [data.astype(np.float32)]

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8 
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

# Save the model.
tflite_model_path = 'mobilenet_v2.tflite'
with open(tflite_model_path, 'wb') as f:
  f.write(tflite_model)